In [1]:
import pandas as pd 

| Name       | Description                                                                       | Data Type | Possible Values                   |
|------------|-----------------------------------------------------------------------------------|-----------|-----------------------------------|
| GEO_ID     | Equivalent to countyFIPS, Federal Information Processing Standards for the county | String    | 500000US01001, 0500000US01003,    |
| Name       | Name of the county and state                                                      | String    | “Guilford County, North Carolina” |
| DP05_0001E | Total population of the county                                                    | Integer   | 55639, 218289, 658615             |
| DP05_0001M | Margin of error of the total population                                           | Integer   | 121, 265, 390                     |
| DP05_0002E | Total male population                                                             | Integer   | 78, 136, 244                      |
| DP05_0002M | Margin of error of the male population                                            | Integer   | 27052, 44598, 311447              |

Continues on for 326 columns. The original dataset has the description of the columns in the second row.

In [4]:
df_county = pd.read_csv('covid_county_population_usafacts.csv')
df_deaths = pd.read_csv('covid_deaths_usafacts.csv')
df_confirmed = pd.read_csv('covid_confirmed_usafacts.csv')
df_super = pd.merge(df_confirmed, df_deaths, on = ['countyFIPS', 'County Name', 
                                                   'State', 'StateFIPS'])
df_super = pd.merge(df_county, df_super, on = ['countyFIPS', 'State'])
df_super.to_csv('covid_super.csv', index=False)

## Enrichment Dataset with COVID Dataset

In [31]:
census = pd.read_csv("census_by_county.csv",low_memory=False)
covid_census = pd.read_csv("covid_county_population_usafacts.csv")

Dropped rows containing Puerto Rico (US territory), empty column at the end, and NAME and GEO_ID (both present in the covid census)

In [19]:
us_census = census[census['GEO_ID'] < '0500000US72001'] 
us_census = us_census.dropna(axis=1, how='all') 
us_census = us_census.drop(['NAME', 'GEO_ID'], axis=1) 

Dropped Statewide unallocated and Grand Prince Cruise Ship resulting in both dataframes having the same number of rows

In [21]:
covid_census = covid_census[covid_census['countyFIPS'] != 0]
covid_census = covid_census.drop(187) 

Concatenates the two dataframe without regard to the indexes

In [25]:
covid_census = covid_census.reset_index(drop=True)
us_census = us_census.reset_index(drop=True)

county_breakdown = pd.concat([covid_census, us_census], axis=1)
county_breakdown.head()

,countyFIPS,County Name,State,population,DP05_0001E,DP05_0001M,DP05_0002E,DP05_0002M,DP05_0003E,DP05_0003M,...,DP05_0085PE,DP05_0085PM,DP05_0086PE,DP05_0086PM,DP05_0087PE,DP05_0087PM,DP05_0088PE,DP05_0088PM,DP05_0089PE,DP05_0089PM
0,1001,Autauga County,AL,55869,55639,*****,27052,167,28587,167,...,1.9,0.7,(X),(X),41959,(X),47.6,0.4,52.4,0.4
1,1003,Baldwin County,AL,223234,218289,*****,105889,253,112400,253,...,1.8,0.4,(X),(X),167986,(X),47.8,0.2,52.2,0.2
2,1005,Barbour County,AL,24686,25026,*****,13156,86,11870,86,...,1.2,0.6,(X),(X),19468,(X),53.2,0.2,46.8,0.2
3,1007,Bibb County,AL,22394,22374,*****,12022,170,10352,170,...,0.4,0.3,(X),(X),17583,(X),53.6,0.4,46.4,0.4
4,1009,Blount County,AL,57826,57755,*****,28677,153,29078,153,...,1.6,0.3,(X),(X),42978,(X),48.9,0.3,51.1,0.3


Repeated the same for the cases of covid confirmed 

In [61]:
census = pd.read_csv("census_by_county.csv",low_memory=False)
covid_confirmed = pd.read_csv("covid_confirmed_usafacts.csv")

us_census = census[census['GEO_ID'] < '0500000US72001'] #Dropped rows with Puerto Rico (US territory)
us_census = us_census.dropna(axis=1, how='all') #Dropped NaN column at the end

covid_confirmed = covid_confirmed[covid_confirmed['countyFIPS'] != 0]

us_census = us_census.reset_index(drop=True)
covid_confirmed = covid_confirmed.reset_index(drop=True)

cases_census = pd.concat([covid_confirmed, us_census], axis = 1)
cases_census.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,DP05_0085PE,DP05_0085PM,DP05_0086PE,DP05_0086PM,DP05_0087PE,DP05_0087PM,DP05_0088PE,DP05_0088PM,DP05_0089PE,DP05_0089PM
0,1001.0,Autauga County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.9,0.7,(X),(X),41959,(X),47.6,0.4,52.4,0.4
1,1003.0,Baldwin County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,0.4,(X),(X),167986,(X),47.8,0.2,52.2,0.2
2,1005.0,Barbour County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.2,0.6,(X),(X),19468,(X),53.2,0.2,46.8,0.2
3,1007.0,Bibb County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.3,(X),(X),17583,(X),53.6,0.4,46.4,0.4
4,1009.0,Blount County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.6,0.3,(X),(X),42978,(X),48.9,0.3,51.1,0.3


Repeat the same for COVID deaths

In [12]:
census = pd.read_csv("census_by_county.csv",low_memory=False)
covid_deaths = pd.read_csv("covid_deaths_usafacts.csv")

us_census = census[census['GEO_ID'] < '0500000US72001'] #Dropped rows with Puerto Rico (US territory)
us_census = us_census.dropna(axis=1, how='all') #Dropped NaN column at the end

covid_deaths = covid_deaths[covid_deaths['countyFIPS'] != 0]

us_census = us_census.reset_index(drop=True)
covid_deaths = covid_deaths.reset_index(drop=True)

death_census = pd.concat([covid_deaths, us_census], axis = 1)
death_census.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,DP05_0085PE,DP05_0085PM,DP05_0086PE,DP05_0086PM,DP05_0087PE,DP05_0087PM,DP05_0088PE,DP05_0088PM,DP05_0089PE,DP05_0089PM
0,1001.0,Autauga County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.9,0.7,(X),(X),41959,(X),47.6,0.4,52.4,0.4
1,1003.0,Baldwin County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.8,0.4,(X),(X),167986,(X),47.8,0.2,52.2,0.2
2,1005.0,Barbour County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.2,0.6,(X),(X),19468,(X),53.2,0.2,46.8,0.2
3,1007.0,Bibb County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4,0.3,(X),(X),17583,(X),53.6,0.4,46.4,0.4
4,1009.0,Blount County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.6,0.3,(X),(X),42978,(X),48.9,0.3,51.1,0.3


## Washington COVID data (for the last 7 days)

Import and select the last seven days and compute the mean 

In [14]:
wa_cases = pd.read_csv("wa_cases.csv")
cases_week = wa_cases.iloc[:,1262:1269]

print("Mean:")
for columns in cases_week:
    print(cases_week.loc[:,columns].mean())

Mean:
31188.743814594425
31189.750078296274
31191.313811462573
31191.974318822424
31192.214531788286
31192.214531788286
31192.214531788286


Import and select the last seven days and compute the mean 

In [21]:
wa_deaths = pd.read_csv("wa_deaths.csv")
deaths_week = wa_deaths.iloc[:,1262:1269]

print("Mean:")
for columns in deaths_week:
    print(deaths_week.loc[:,columns].mean())

Mean:
345.8083307234576
345.819918571876
345.825555903539
345.75070466645786
345.7563419981209
345.7563419981209
345.7563419981209


The cases and deaths for the last seven days in Washington are stable.